# 🍏 `azure-ai-projects` 및 `azure-ai-inference`를 사용한 관찰 가능성 및 추적 데모 🍎

이 **건강 및 피트니스** 테마 노트북에서는 **관측 가능성** 및 **추적**을 설정하는 방법을 살펴봅니다:

1. `AIProjectClient`를 사용한 **기본 LLM 호출**.
2. **에이전트**(예: 헬스 리소스 에이전트)를 사용한 **다단계** 상호작용.
3. **console**(stdout) 또는 **OTLP 엔드포인트**(예: **Prompty** 또는 **Aspire**)를 통해 로컬 사용량 **추적**.
4. 해당 **트레이스**를 **Azure Monitor**(애플리케이션 인사이트)로 전송하여 **Azure AI Foundry**에서 볼 수 있도록 합니다.

> **고지 사항**: 이것은 AI 및 통합 가시성에 대한 재미있는 데모입니다! 코드 또는 프롬프트에서 운동, 식단 또는 건강 루틴에 대한 모든 참조는 순전히 **교육적** 목적으로만 사용됩니다. 건강에 대한 조언은 항상 전문가와 상의하세요.

## 내용
1. **초기화**: 환경 설정, 클라이언트 생성.
2. **기본 LLM 호출**: 모델 completion 검색에 대한 빠른 데모.
3. **연결**: 프로젝트 연결 목록 표시.
4. **가시성 및 추적**
   - **콘솔/로컬** 추적
   - **Prompty / Aspire**: 추적을 로컬 OTLP 엔드포인트로 파이프하기
   - **Azure Monitor** 추적: 애플리케이션 인사이트에 연결
   - Azure AI Foundry에서 추적 **확인**하기
5. **에이전트 기반 예제**:
   - 샘플 문서를 참조하여 간단한 “상태 리소스 에이전트” 만들기.
   - 추적을 사용한 다중 턴 대화.
   - 정리.


<img src="./seq-diagrams/1-observability.png" width="50%"/>

## 1. 초기화 및 설정
**전제 조건**:
- 환경 변수 `PROJECT_CONNECTION_STRING`(및 선택적으로 `MODEL_DEPLOYMENT_NAME`)이 포함된 `.env` 파일.
- 추론 및 에이전트 생성을 수행할 수 있는 Azure AI Foundry의 역할/권한.
- `azure-ai-projects`, `azure-ai-inference`, `opentelemetry` 패키지가 설치된 로컬 환경.

**할 일**:
- 환경 변수를 로드합니다.
- `AIProjectClient`를 초기화.
- 모델(예: `gpt-4o`)과 대화할 수 있는지 확인합니다.

In [ ]:
import os
import sys
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, CompletionsFinishReason

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / '.env'  # Adjust path as needed
load_dotenv(env_path)

connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
if not connection_string:
    raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env.")

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string
    )
    print("✅ Successfully created AIProjectClient!")
except Exception as e:
    print(f"❌ Error creating AIProjectClient: {e}")

## 2. 기본 LLM 호출
모든 것이 제대로 작동하는지 확인하기 위해 **빠른** 채팅 완료 요청을 할 것입니다. 간단한 질문을 하겠습니다: "How many feet are in a mile?"

In [ ]:
try:
    # Create a ChatCompletions client
    inference_client = project_client.inference.get_chat_completions_client()
    # Default to "gpt-4o" if no env var is set
    model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

    user_question = "How many feet are in a mile?"
    response = inference_client.complete(
        model=model_name,
        messages=[UserMessage(content=user_question)]
    )
    print("\n💡Response:")
    print(response.choices[0].message.content)
    print("\nFinish reason:", response.choices[0].finish_reason)

except Exception as e:
    print("❌ Could not complete the chat request:", e)

## 3. 연결 나열하고 검사
프로젝트에 있는 **연결**을 확인하세요. Azure OpenAI 또는 기타 리소스 첨부 파일일 수 있습니다. 데모를 위해 여기에 나열해 보겠습니다.

In [ ]:
from azure.ai.projects.models import ConnectionType

all_conns = project_client.connections.list()
print(f"🔎 Found {len(all_conns)} total connections.")
for idx, c in enumerate(all_conns):
    print(f"{idx+1}) Name: {c.name}, Type: {c.connection_type}, Endpoint: {c.endpoint_url}")

# Filter for Azure OpenAI connections
aoai_conns = project_client.connections.list(connection_type=ConnectionType.AZURE_OPEN_AI)
print(f"\n🌀 Found {len(aoai_conns)} Azure OpenAI connections:")
for c in aoai_conns:
    print(f"   -> {c.name}")

# Get default connection of type AZURE_AI_SERVICES
default_conn = project_client.connections.get_default(connection_type=ConnectionType.AZURE_AI_SERVICES,
                                                     include_credentials=False)
if default_conn:
    print("\n⭐ Default Azure AI Services connection:")
    print(default_conn)
else:
    print("No default connection found for Azure AI Services.")

# 4. 가시성 및 추적

예를 들어 LLM 통화에서 **텔레메트리를 수집**하려고 합니다. 예를 들어:
- 요청의 타임스탬프.
- 지연시간.
- 잠재적 오류.
- 선택 사항으로 실제 프롬프트 및 응답(콘텐츠 녹화를 활성화한 경우).

설정 방법은 다음과 같습니다:
1. **콘솔** 또는 로컬 OTLP 엔드포인트 계측.
2. 애플리케이션 인사이트를 사용한 **Azure Monitor** 계측.
3. Azure AI Foundry의 포털에서 추적 **보기**.

## 4.1 로컬 콘솔 디버깅
계측 패키지를 설치하고 활성화합니다. 그런 다음 빠른 채팅 호출을 통해 **stdout**에 로그가 표시되는지 확인합니다.


**참고**: 더 고급 로컬 대시보드를 보고 싶으시다면 다음을 수행합니다::
- [Prompty](https://github.com/microsoft/prompty) 사용.
- [Aspire Dashboard](https://learn.microsoft.com/dotnet/aspire/fundamentals/dashboard/standalone?tabs=bash) 를 사용하여 OTLP 추적을 시각화합니다.

In [ ]:
# You only need to install these once.
!pip install opentelemetry-instrumentation-openai-v2 opentelemetry-exporter-otlp-proto-grpc

### 4.1.1 Azure AI 추론을 위한 OpenTelemetry 활성화
환경 변수를 설정하여 다음을 보장합니다:
1. **프롬프트 콘텐츠**가 캡처됩니다(선택 사항!).
2. **Azure SDK**가 추적 구현으로 OpenTelemetry를 사용합니다.
3. `AIInferenceInstrumentor().instrument()`를 호출하여 계측을 패치하고 활성화합니다.

In [ ]:
import os
from azure.ai.inference.tracing import AIInferenceInstrumentor

# (Optional) capture prompt & completion contents in traces
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"  # or 'false'

# Let the Azure SDK know we want to use OpenTelemetry
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"

# Instrument the Azure AI Inference client library
AIInferenceInstrumentor().instrument()
print("✅ Azure AI Inference instrumentation enabled.")

### 4.1.2 콘솔 또는 로컬 OTLP로 트레이스 지정
가장 간단한 방법은 **stdout**으로 파이프하는 것입니다. **Prompty** 또는 **Aspire**로 보내려면 로컬 OTLP 엔드포인트 URL(일반적으로 `"http://localhost:4317"` 또는 이와 유사한 URL)을 지정하세요.

In [ ]:
project_client.telemetry.enable(destination=sys.stdout)
# Or, to send to a local OTLP collector (Prompty/Aspire), do:
#   project_client.telemetry.enable(destination="http://localhost:4317")

try:
    local_client = project_client.inference.get_chat_completions_client()
    user_prompt = "What's a simple 5-minute warmup routine?"
    local_resp = local_client.complete(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        messages=[UserMessage(content=user_prompt)]
    )
    print("\n🤖 Response:", local_resp.choices[0].message.content)
except Exception as exc:
    print(f"❌ Error in local-tracing example: {exc}")

## 4.2 Azure Monitor 추적 (애플리케이션 인사이트)
이제 **애플리케이션 인사이트**에 추적을 설정하여 로그를 **Azure AI Foundry** **추적** 페이지로 전달합니다.

**단계**:
1. 이제 AI Foundry의 프로젝트 **Tracing**탭으로 이동하요, **애플리케이션 인사이트** 자원을 붙이거나 생성합니다.
2. 코드에서 `project_client.telemetry.get_connection_string()`을 호출하여 계측 키를 검색합니다.
3. 해당 연결과 함께 `azure.monitor.opentelemetry.configure_azure_monitor(...)`를 사용합니다.
4. 추론 호출을 수행하면 -> 로그가 Foundry 포털(및 Azure Monitor 자체)에 표시됩니다.


In [ ]:
%pip install azure-monitor-opentelemetry

In [ ]:
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.inference.models import UserMessage

app_insights_conn_str = project_client.telemetry.get_connection_string()
if app_insights_conn_str:
    print("🔧 Found App Insights connection string, configuring...")
    configure_azure_monitor(connection_string=app_insights_conn_str)
    # Optionally add more instrumentation (for openai or langchain):
    project_client.telemetry.enable()
    
    # Let's do a test call that logs to AI Foundry's Tracing page
    try:
        with project_client.inference.get_chat_completions_client() as client:
            prompt_msg = "Any easy at-home cardio exercise recommendations?"
            response = client.complete(
                model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
                messages=[UserMessage(content=prompt_msg)]
            )
            print("\n🤖 Response (logged to App Insights):")
            print(response.choices[0].message.content)
    except Exception as e:
        print("❌ Chat completions with Azure Monitor example failed:", e)
else:
    print("No Application Insights connection string is configured in this project.")

### 4.3 Azure AI Foundry에서 트레이스 보기
위의 코드를 실행한 후:
1. AI Foundry 프로젝트로 이동합니다.
2. 사이드바에서 **Tracing**을 클릭합니다.
3. 호출에서 로그를 봅니다.
4. 필요에 따라 로그를 필터링, 확장 또는 탐색합니다.

또한 고급 대시보드가 필요한 경우, 파운드리에서 **애플리케이션 인사이트** 리소스를 열 수 있습니다. 앱 인사이트 포털에서는 **엔드투엔드 트랜잭션** 세부 정보, 쿼리 로그 등과 같은 추가 기능을 이용할 수 있습니다.

# 5. 에이전트 기반 예제
이제 레시피나 가이드라인에 대한 샘플 문서를 참조하는 **헬스 리소스 에이전트**를 만든 다음 시연해 보겠습니다:
1. 지침이 포함된 에이전트 만들기.
2. 대화 스레드 만들기.
3. **observability**를 사용 설정한 상태에서 다단계 쿼리 실행하기.
4. 선택적으로 마지막에 리소스 정리하기.

> 에이전트 접근 방식은 보다 정교한 대화 흐름이나 **도구 사용**(예: 파일 검색)을 원할 때 유용합니다.

## 5.1 샘플 파일 및 벡터 저장소 만들기
레시피/가이드라인에 대한 더미 `.md` 파일을 만든 다음 에이전트가 시맨틱 검색을 할 수 있도록 **벡터 스토어**에 푸시하겠습니다.

(*이 부분은 간략하게 요약한 것입니다. 자세한 내용은 [the other file-search tutorial] 을 참고하세요.)

In [ ]:
from azure.ai.projects.models import (
    FileSearchTool,
    FilePurpose,
    MessageTextContent,
    MessageRole
)

def create_sample_files():
    """Create some local .md files with sample text."""
    recipes_md = (
        """# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"""
    )

    guidelines_md = (
        """# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"""
    )

    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]

sample_files = create_sample_files()

def create_vector_store(files, store_name="my_health_resources"):
    try:
        uploaded_ids = []
        for fp in files:
            upl = project_client.agents.upload_file_and_poll(
                file_path=fp,
                purpose=FilePurpose.AGENTS  # Add FilePurpose.AGENTS here
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Create vector store from these file IDs
        vs = project_client.agents.create_vector_store_and_poll(
            file_ids=uploaded_ids,
            name=store_name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []

vector_store, file_ids = None, []
if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, store_name="health_resources_example")

## 5.2 상태 리소스 에이전트 생성
벡터 저장소를 참조하는 **FileSearchTool**를 생성한 다음, 에이전트에 필요한 지침을 사용하여 에이전트를 생성하겠습니다:
1. 면책 조항을 제공합니다.
2. 일반적인 영양 또는 레시피 팁을 제공합니다.
3. 가능한 경우 출처를 인용합니다.
4. 보다 심층적인 의학적 조언을 위해 전문가 상담을 권장합니다.


In [ ]:
from azure.ai.projects.models import FileSearchTool, FilePurpose
from azure.ai.projects.models import ConnectionType, MessageTextContent, MessageRole

def create_health_agent(vs_id):
    try:
        # The tool references our vector store so the agent can search it
        file_search_tool = FileSearchTool(vector_store_ids=[vs_id])
        
        instructions = """
            You are a health resource advisor with access to dietary and recipe files.
            You:
            1. Always present disclaimers (you're not a medical professional)
            2. Provide references to files when possible
            3. Focus on general nutrition or recipe tips.
            4. Encourage professional consultation for more detailed advice.
        """

        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="health-search-agent",
            instructions=instructions,
            tools=file_search_tool.definitions,
            tool_resources=file_search_tool.resources
        )
        print(f"🎉 Created agent '{agent.name}' with ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating health agent: {e}")
        return None

health_agent = None
if vector_store:
    health_agent = create_health_agent(vector_store.id)

## 5.3 에이전트 사용하기
새 대화 **쓰레드**를 만들어 상담원에게 몇 가지 질문을 해 보겠습니다. 각 단계를 추적할 수 있도록 이미 구성한 **observability** 설정을 사용하겠습니다.

In [ ]:
def create_thread():
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created new thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Could not create thread: {e}")
        return None

def ask_question(thread_id, agent_id, user_question):
    try:
        # 1) Add user message
        msg = project_client.agents.create_message(
            thread_id=thread_id,
            role="user",
            content=user_question
        )
        print(f"User asked: '{user_question}'")
        # 2) Create & process a run
        run = project_client.agents.create_and_process_run(
            thread_id=thread_id,
            assistant_id=agent_id
        )
        print(f"Run finished with status: {run.status}")
        if run.last_error:
            print("Error details:", run.last_error)
        return run
    except Exception as e:
        print(f"❌ Error asking question: {e}")
        return None

if health_agent:
    thread = create_thread()
    if thread:
        # Let's ask a few sample questions
        queries = [
            "Could you suggest a gluten-free lunch recipe?",
            "Show me some heart-healthy meal ideas.",
            "What guidelines do you have for someone with diabetes?"
        ]
        for q in queries:
            ask_question(thread.id, health_agent.id, q)


### 5.3.1 대화 보기
대화 메시지를 검색하여 상담원이 어떻게 응답했는지, 파일 내용을 인용했는지 등을 확인할 수 있습니다.

In [ ]:
def display_thread(thread_id):
    try:
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation:")
        for m in reversed(messages.data):
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    print(f"[{m.role.upper()}]: {last_content.text.value}\n")

        print("\n📎 Checking for citations...")
        for c in messages.file_citation_annotations:
            print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")
    except Exception as e:
        print(f"❌ Could not display thread: {e}")

# If we created a thread above, let's read it
if health_agent and thread:
    display_thread(thread.id)

# 6. 정리
원하는 경우 벡터 저장소, 파일 및 에이전트를 제거하여 깔끔하게 정리할 수 있습니다. (프로덕션에서는 계속 유지할 수도 있습니다.)

In [ ]:
def cleanup_resources():
    try:
        if 'vector_store' in globals() and vector_store:
            project_client.agents.delete_vector_store(vector_store.id)
            print("🗑️ Deleted vector store.")

        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.delete_file(fid)
            print("🗑️ Deleted uploaded files.")

        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health agent.")

        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")
    except Exception as e:
        print(f"❌ Error cleaning up: {e}")


cleanup_resources()

# 🎉 마무리
다음을 시연해 보았습니다:
1. `AIProjectClient`로 **기본 LLM 호출**.
2. Azure AI Foundry 프로젝트에서 **연결 나열하기**.
3. 로컬(콘솔, OTLP 엔드포인트) 및 클라우드(앱 인사이트) 컨텍스트에서 **Observability & tracing**.
4. 샘플 문서 검색을 위해 벡터 저장소를 사용하는 빠른 **Agent** 시나리오.

## 다음 단계
- Azure AI Foundry 포털에서 **Tracing** 탭을 확인하여 로그를 확인합니다.
- 애플리케이션 인사이트에서 고급 쿼리를 살펴봅니다.
- 로컬 원격 분석 대시보드의 경우 [Prompty](https://github.com/microsoft/prompty) 또는 [Aspire](https://learn.microsoft.com/dotnet/aspire/) 를 사용하세요.
- 이 접근 방식을 **프로덕션** GenAI 파이프라인에 통합하세요!

> 🏋️ **건강 알림**: LLM의 제안은 데모용으로만 제공됩니다. 실제 건강 관련 결정은 전문가와 상의하세요.

행복한 관찰과 추적되세요! 🎉